In [ ]:
# !wget https://data.bev.gv.at/download/DOP/20250415/2024350_Mosaik_RGB.tif -O "/content/drive/MyDrive/Colab Notebooks/mastarbeit/in_data/2024350_Mosaik_RGB.tif"

In [ ]:
!pip install rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 22.1 MB/s eta 0:00:00


In [ ]:
!pip install osmnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.2/100.2 kB 1.7 MB/s eta 0:00:00


In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
import rasterio
from shapely.geometry import box, Polygon, MultiPolygon, LineString, Point
from rasterio.windows import Window
import osmnx as ox
import os

# It's good practice to configure OSMnx once
ox.settings.use_cache = True
ox.settings.log_console = False # Quieter output

def download_osm_data_optimized(
    osm_class_definitions: dict,
    hierarchy: list,
    in_raster_path: str,
    out_gdf_base_path: str,
    num_subregions_x: int = 3,
    num_subregions_y: int = 3
):
    """
    Optimized version of the OSM data download and processing function.
    - Uses vectorized operations for buffering and overlay.
    - Assembles GeoDataFrames efficiently.
    - Performs a vectorized hierarchical overlay.
    - Splits the input raster's AOI into subregions and processes each.
    """
    try:
        with rasterio.open(in_raster_path) as src:
            width_orig = src.width
            height_orig = src.height
            transform_orig = src.transform
            crs_orig = src.crs
            print(f"Original Raster Width: {width_orig}, Height: {height_orig}")
            print(f"Original Raster Transform: {transform_orig}")
            print(f"Original Raster CRS: {crs_orig}")
    except Exception as e:
        print(f"Error loading raster: {e}. Exiting.")
        return

    # --- Subregion Splitting Logic ---
    print(f"\n--- Splitting AOI into {num_subregions_y} rows x {num_subregions_x} columns ---")

    subregion_width_pixels = width_orig // num_subregions_x
    subregion_height_pixels = height_orig // num_subregions_y

    all_subregion_gdfs_orig = []

    for row_idx in range(num_subregions_y):
        for col_idx in range(num_subregions_x):
            col_off = col_idx * subregion_width_pixels
            row_off = row_idx * subregion_height_pixels

            current_subregion_width = subregion_width_pixels
            if col_idx == num_subregions_x - 1:
                current_subregion_width = width_orig - col_off

            current_subregion_height = subregion_height_pixels
            if row_idx == num_subregions_y - 1:
                current_subregion_height = height_orig - row_off

            window = Window(col_off, row_off, current_subregion_width, current_subregion_height)

            left, bottom, right, top = src.window_bounds(window)
            subregion_polygon_orig = box(left, bottom, right, top)

            subregion_aoi_gdf_orig = gpd.GeoDataFrame(geometry=[subregion_polygon_orig], crs=crs_orig)
            all_subregion_gdfs_orig.append(subregion_aoi_gdf_orig)

            print(f"  Subregion (Row {row_idx+1}, Col {col_idx+1}):")
            print(f"    Pixel Window: ({col_off}, {row_off}) to ({col_off + current_subregion_width}, {row_off + current_subregion_height})")
            print(f"    Native CRS Bounds: ({left:.2f}, {bottom:.2f}, {right:.2f}, {top:.2f})")

    # --- Process Each Subregion Individually ---
    for i, aoi_gdf_orig in enumerate(all_subregion_gdfs_orig):
        subregion_name = f"subregion_{i+1:02d}_row{row_idx+1}_col{col_idx+1}"
        print(f"\n--- Processing {subregion_name} ---")

        aoi_polygon_orig = aoi_gdf_orig.geometry.iloc[0]
        aoi_bounds_orig = aoi_polygon_orig.bounds

        aoi_wgs84 = aoi_gdf_orig.to_crs(epsg=4326)
        aoi_bbox_wgs84 = aoi_wgs84.total_bounds

        print(f"AOI CRS: {crs_orig}, AOI Bounds (Native CRS): {aoi_bounds_orig}")
        print(f"AOI Bounds (WGS84 for OSMnx): {aoi_bbox_wgs84}")

        # --- Part 1: Vectorized Data Ingestion and Pre-processing for current subregion ---
        all_processed_gdfs = []
        print(f"\n--- Starting OSM Feature Extraction for {subregion_name} ---")

        for temp_class_id, params in osm_class_definitions.items():
            final_class_id = params.get("final_class_id", temp_class_id)
            class_name = params.get("name", f"Class {final_class_id}")
            print(f"Processing: {class_name} (Class ID: {final_class_id}) for {subregion_name}")

            try:
                gdf = ox.features.features_from_bbox(bbox=aoi_bbox_wgs84, tags=params['tags'])

                if gdf.empty or 'geometry' not in gdf.columns:
                    print(f"   No features found for {class_name} in {subregion_name}.")
                    continue

                gdf = gdf.explode(index_parts=False, ignore_index=True)
                gdf = gdf[gdf.geometry.geom_type.isin(params["geom_types"])]

                if gdf.empty:
                    print(f"   No features of desired types found for {class_name} in {subregion_name}.")
                    continue

                gdf_proj = gdf.to_crs(crs_orig)
                gdf_proj['class_id'] = final_class_id

                if "buffer_m" in params and params["buffer_m"]:
                    def get_buffer_dist(row_geom_type):
                        return params["buffer_m"].get(row_geom_type, 0)

                    buffer_distances = gdf_proj.geometry.geom_type.apply(get_buffer_dist)

                    rows_to_buffer = buffer_distances > 0
                    if rows_to_buffer.any():
                        gdf_proj.loc[rows_to_buffer, 'geometry'] = gdf_proj.loc[rows_to_buffer].buffer(buffer_distances[rows_to_buffer])

                # Apply make_valid to the entire GeoSeries
                gdf_proj.geometry = gdf_proj.geometry.make_valid()

                gdf_proj = gdf_proj[['class_id', 'geometry']]

                all_processed_gdfs.append(gdf_proj)
                print(f"   Collected {len(gdf_proj)} features for {class_name} in {subregion_name}.")

            except Exception as e:
                print(f"   ERROR processing {class_name} for {subregion_name}: {e}")

        if not all_processed_gdfs:
            print(f"No OSM features were collected for {subregion_name}. Skipping output.")
            continue

        print(f"\n--- Consolidating all features into one GeoDataFrame for {subregion_name} ---")
        osm_data_gdf = pd.concat(all_processed_gdfs, ignore_index=True)
        osm_data_gdf = osm_data_gdf[~osm_data_gdf.geometry.is_empty]
        osm_data_gdf.geometry = osm_data_gdf.geometry.make_valid() # Apply make_valid here too
        print(f"Total consolidated features for {subregion_name}: {len(osm_data_gdf)}")

        # --- Part 2: Vectorized Hierarchical Overlay for current subregion ---
        print(f"\n--- Starting Vectorized Hierarchical Overlay for {subregion_name} ---")

        final_pieces = []
        higher_priority_union = None

        for class_id in hierarchy:
            current_layer = osm_data_gdf[osm_data_gdf['class_id'] == class_id].copy() # Use .copy() to avoid SettingWithCopyWarning

            if current_layer.empty:
                continue

            print(f"Processing class: {class_id} ({len(current_layer)} features) for {subregion_name}")

            # Apply make_valid to the entire GeoSeries before dissolving
            current_layer.geometry = current_layer.geometry.make_valid()
            current_layer_dissolved = current_layer.dissolve()

            if current_layer_dissolved.empty or current_layer_dissolved.geometry.is_empty.all():
                continue

            # This will be a GeoSeries, and .iloc[0] extracts the shapely geometry
            processed_layer_geom_series = current_layer_dissolved.geometry
            # We must apply make_valid to the GeoSeries, not the individual geometry yet
            processed_layer_geom_series = processed_layer_geom_series.make_valid()

            # Now extract the actual geometry object
            processed_layer_geom = processed_layer_geom_series.iloc[0]


            if higher_priority_union is not None:
                # Ensure higher_priority_union is valid (it should be if previous steps are clean)
                # and then perform the difference.
                # Use geometry.difference on a GeoSeries for robustness
                processed_layer_geom = gpd.GeoSeries([processed_layer_geom]).difference(gpd.GeoSeries([higher_priority_union]))
                if not processed_layer_geom.empty:
                    processed_layer_geom = processed_layer_geom.iloc[0] # Extract the resulting geometry
                else:
                    processed_layer_geom = Polygon() # Set to empty polygon if difference results in empty

            if not processed_layer_geom.is_empty:
                processed_gdf = gpd.GeoDataFrame({'class_id': [class_id], 'geometry': [processed_layer_geom]}, crs=crs_orig)
                final_pieces.append(processed_gdf)

            # Update higher_priority_union with the *validated* geometry of the current layer
            # Again, ensure make_valid is applied to the GeoSeries then extract
            union_candidate_geom = current_layer_dissolved.geometry.iloc[0]
            union_candidate_geom = gpd.GeoSeries([union_candidate_geom]).make_valid().iloc[0]


            if higher_priority_union is None:
                higher_priority_union = union_candidate_geom
            else:
                # Use geometry.union on GeoSeries for robustness
                higher_priority_union = gpd.GeoSeries([higher_priority_union]).union(gpd.GeoSeries([union_candidate_geom]))
                if not higher_priority_union.empty:
                    higher_priority_union = higher_priority_union.iloc[0]
                else:
                    higher_priority_union = Polygon() # Set to empty polygon if union results in empty

        if not final_pieces:
            print(f"Processing resulted in an empty GeoDataFrame for {subregion_name}. Skipping output.")
            continue

        final_gdf = pd.concat(final_pieces, ignore_index=True)

        print(f"\n--- Finalizing output: exploding to single parts and clipping for {subregion_name} ---")
        final_gdf = final_gdf.explode(index_parts=False, ignore_index=True)

        final_gdf = gpd.clip(final_gdf, aoi_gdf_orig)
        final_gdf = final_gdf[~final_gdf.geometry.is_empty]
        final_gdf.geometry = final_gdf.geometry.make_valid() # Final validation before saving

        # --- Exporting GDF for the current subregion ---
        output_folder = os.path.dirname(out_gdf_base_path)
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)
            print(f"Created output directory: {output_folder}")

        subregion_out_path = os.path.join(output_folder, f"{os.path.basename(out_gdf_base_path).replace('.gpkg', '')}_{subregion_name}.gpkg")

        try:
            final_gdf.to_file(subregion_out_path, driver="GPKG")
            print(f"\nSuccess! Saved {len(final_gdf)} final features for {subregion_name} to {subregion_out_path}")
        except Exception as e:
            print(f"ERROR: Could not save GeoDataFrame for {subregion_name} to {subregion_out_path}: {e}")

    print("\n--- All subregions processed ---")


In [ ]:
osm_class_definitions = {
    # Urban/Built-up
    1: {"name": "Residential Building", "tags": {"building": ["residential", "house", "apartments", "semidetached_house", "terrace"]}, "geom_types": ["Polygon", "MultiPolygon"]},
    2: {"name": "Commercial/Office Building", "tags": {"building": ["commercial", "office", "retail"]}, "geom_types": ["Polygon", "MultiPolygon"]},
    3: {"name": "Industrial Building", "tags": {"building": ["industrial", "warehouse"]}, "geom_types": ["Polygon", "MultiPolygon"]},
    4: {"name": "Unspecified Building", "tags": {"building": "yes"}, "geom_types": ["Polygon", "MultiPolygon"]}, # Note: Report mentions 'building=yes'

    # Transportation
    # For roads, the report emphasizes using the 'surface' tag.
    # This creates a challenge if 'surface' is not present.
    # See "Weaknesses and Potential Barriers" section for discussion.
    # For now, these definitions strictly follow the "Paved/Unpaved" logic based on presence of surface tag.
    5: {"name": "Paved Road (Motorway/Trunk)", "tags": {"highway": ["motorway", "trunk", "primary"], "surface": ["asphalt", "concrete", "paved"]},
        "geom_types": ["LineString", "MultiLineString"], "buffer_m": {"LineString": 5, "MultiLineString": 5}},
    6: {"name": "Paved Road (Other)", "tags": {"highway": ["secondary", "tertiary", "residential", "service", "unclassified", "living_street"], "surface": ["asphalt", "concrete", "paved"]}, # Added 'unclassified', 'living_street'
        "geom_types": ["LineString", "MultiLineString"], "buffer_m": {"LineString": 3, "MultiLineString": 3}},
    7: {"name": "Unpaved Road", "tags": {"highway": ["track", "unclassified", "residential", "service", "path"], "surface": ["dirt", "gravel", "unpaved", "ground", "sand"]}, # Added more highway types and surface types
        "geom_types": ["LineString", "MultiLineString"], "buffer_m": {"LineString": 1.5, "MultiLineString": 1.5}},

    # Class 8: Parking Area. Report: amenity=parking (Area), highway=service + service=parking_aisle (Line)
    # We will handle this by fetching areas first, then lines, and assigning them the same class_id.
    8: {"name": "Parking Area (Area)", "tags": {"amenity": "parking"}, "geom_types": ["Polygon", "MultiPolygon"]},
    801: {"name": "Parking Area (Aisles)", "tags": {"highway": "service", "service":"parking_aisle"}, # Custom temp ID for merging later
          "geom_types": ["LineString", "MultiLineString"], "buffer_m": {"LineString": 2.5, "MultiLineString": 2.5}, "final_class_id": 8}, # Assumed buffer for aisles

    9: {"name": "Railway (Area)", "tags": {"landuse": "railway"}, "geom_types": ["Polygon", "MultiPolygon"]},
    901: {"name": "Railway (Track)", "tags": {"railway": ["rail", "light_rail", "tram", "subway"]}, # More railway types
          "geom_types": ["LineString", "MultiLineString"], "buffer_m": {"LineString": 3, "MultiLineString": 3}, "final_class_id": 9}, # Buffer for tracks

    # Natural/Vegetation
    10: {"name": "Forest/Woodland", "tags": {"natural": "wood", "landuse": "forest"}, "geom_types": ["Polygon", "MultiPolygon"]},
    11: {"name": "Grassland/Mown Grass", "tags": {"natural": "grassland", "landuse": ["grass", "meadow"]}, "geom_types": ["Polygon", "MultiPolygon"]},
    12: {"name": "Heath/Scrub", "tags": {"natural": ["heath", "scrub"]}, "geom_types": ["Polygon", "MultiPolygon"]},
    13: {"name": "Bare Earth", "tags": {"natural": ["bare_rock", "mud", "sand", "shingle", "scree"]}, "geom_types": ["Polygon", "MultiPolygon"]}, # Scree also added here as per report description for bare earth
    14: {"name": "Wetland", "tags": {"natural": "wetland" # wetland=* is too broad for direct query, often natural=wetland is the main polygon.
                                     # Specific types can be refined later if needed.
                                    }, "geom_types": ["Polygon", "MultiPolygon"]},
    15: {"name": "Cliff", "tags": {"natural": "cliff"}, "geom_types": ["Polygon", "MultiPolygon", "LineString", "MultiLineString"], "buffer_m": {"LineString": 2, "MultiLineString": 2}}, # Assumed buffer for linear cliffs
    16: {"name": "Glacier", "tags": {"natural": "glacier"}, "geom_types": ["Polygon", "MultiPolygon"]},
    17: {"name": "Scree/Talus", "tags": {"natural": "scree"}, "geom_types": ["Polygon", "MultiPolygon"]}, # Already partly in 13, can be duplicative or allow specific mapping.

    # Water
    18: {"name": "River/Large Waterbody", "tags": {"natural": "water", "water": ["river", "lake", "reservoir", "ocean"]}, "geom_types": ["Polygon", "MultiPolygon"]},
    19: {"name": "Stream/Canal/Drainage Ditch", "tags": {"waterway": ["stream", "canal", "drain", "ditch"]},
         "geom_types": ["LineString", "MultiLineString"], "buffer_m": {"LineString": 1, "MultiLineString": 1}},
    20: {"name": "Swimming Pool", "tags": {"leisure": "swimming_pool", "amenity": "swimming_pool"}, # amenity=swimming_pool is less common
         "geom_types": ["Polygon", "MultiPolygon", "Point", "MultiPoint"], "buffer_m": {"Point": 3, "MultiPoint": 3}}, # Buffer for point representation

    # Agricultural
    21: {"name": "Agricultural Field (Crops)", "tags": {"landuse": ["farmland", "paddy"]}, "geom_types": ["Polygon", "MultiPolygon"]},
    22: {"name": "Orchard", "tags": {"landuse": "orchard"}, "geom_types": ["Polygon", "MultiPolygon"]},
    23: {"name": "Vineyard", "tags": {"landuse": "vineyard"}, "geom_types": ["Polygon", "MultiPolygon"]},
    24: {"name": "Allotments/Community Garden", "tags": {"landuse": "allotments"}, "geom_types": ["Polygon", "MultiPolygon"]},
    25: {"name": "Farmyard", "tags": {"landuse": "farmyard"}, "geom_types": ["Polygon", "MultiPolygon"]},
    26: {"name": "Greenhouse Horticulture", "tags": {"landuse": "greenhouse_horticulture", "building":"greenhouse"}, "geom_types": ["Polygon", "MultiPolygon"]}, # Combining from report and original notebook

    # Infrastructure/Utilities
    27: {"name": "Power Transmission Tower", "tags": {"power": "tower"}, "geom_types": ["Point", "MultiPoint", "Polygon", "MultiPolygon"], # Some towers might be mapped as small areas
         "buffer_m": {"Point": 6, "MultiPoint": 6}},
    28: {"name": "Windmill", "tags": {"man_made": "wind_turbine", "building": "windmill"}, # man_made=wind_turbine is more common for modern ones
         "geom_types": ["Point", "MultiPoint", "Polygon", "MultiPolygon"], "buffer_m": {"Point": 9, "MultiPoint": 9}},
    29: {"name": "Substation", "tags": {"power": "substation"}, "geom_types": ["Polygon", "MultiPolygon"]},
    30: {"name": "Aerodrome", "tags": {"aeroway": ["aerodrome", "runway", "taxiway", "apron"]}, # Aerodrome is often an area, others can be areas or lines
         "geom_types": ["Polygon", "MultiPolygon", "LineString", "MultiLineString"], "buffer_m": {"LineString": 12.5, "MultiLineString": 12.5}}, # Example buffer for linear aeroway features if not areas
    31: {"name": "Quarry", "tags": {"landuse": "quarry"}, "geom_types": ["Polygon", "MultiPolygon"]},

    # Other Developed
    32: {"name": "Cemetery/Graveyard", "tags": {"landuse": "cemetery", "amenity": "grave_yard"}, "geom_types": ["Polygon", "MultiPolygon"]},
    33: {"name": "Recreational Ground/Park", "tags": {"leisure": "park", "landuse": "recreation_ground"}, "geom_types": ["Polygon", "MultiPolygon"]},
    34: {"name": "Sports Pitch/Court", "tags": {"leisure": "pitch"}, "geom_types": ["Polygon", "MultiPolygon"]},

    # Suggested classes for roads with unknown surface (see "Weaknesses" section for approval)
    # 905: {"name": "Paved Road (Motorway/Trunk) - Surface Unknown", "tags": {"highway": ["motorway", "trunk"]}, "exclude_tags":{"surface": True}, # Custom logic needed for exclude
    #      "geom_types": ["LineString", "MultiLineString"], "buffer_m": {"LineString": 5, "MultiLineString": 5}, "needs_review": True},
}

hierarchy = [30, 2, 3, 1, 4, 26, 27, 28, 29, 5, 6, 10, 7, 8, 9, 32, 34, 16, 17, 18, 19, 20, 31, 23, 22, 24, 25, 15, 21, 12, 13, 14, 11, 33]
# raster_path = r"C:\Users\moritz.langer\Downloads\2024350_Mosaik_NIR.tif" # Replace with your actual raster file
raster_file = raster_path = r"/content/drive/MyDrive/Colab Notebooks/mastarbeit/in_data/2024350_Mosaik_RGB.tif"
gdf_out_path = r"/content/drive/MyDrive/Colab Notebooks/mastarbeit/in_data/merged_training_data_1.gpkg"
invekos_gdf_path = r"/content/drive/MyDrive/Colab Notebooks/mastarbeit/in_data/invekos_clipped.gpkg"

In [ ]:
output_gpkg_base = r"C:\Users\moritz.langer\Documents\Masterarbeit_Orthophoto_Segmentierung_Klassifikation\Data_Preparation\Traingebiet_1"

# Ensure the output directory exists for the base path
output_dir = os.path.dirname(output_gpkg_base)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# download_osm_data_optimized(
#     osm_class_definitions=osm_class_definitions,
#     hierarchy=hierarchy,
#     in_raster_path=raster_file,
#     out_gdf_base_path=output_gpkg_base,
#     num_subregions_x=3,
#     num_subregions_y=3
# )

FileNotFoundError: [Errno 2] No such file or directory: ''

In [ ]:
osm_gdf = gpd.read_file(gdf_out_path)

In [ ]:
# Broader Mapping Dictionary
# Keys: SNAR_CODE (int or str)
# Values: FINAL_CLASS_ID (int)

# Default ID for INVEKOS features that don't fit a specific rule below,
# but are clearly agricultural. We'll map them to a broad agricultural category.
# If a SNAR_CODE is truly unclassifiable or represents something non-agricultural
# that doesn't fit OSM, it might get ID_INV_UNKNOWN.
ID_INV_ARABLE_BROAD = 35
ID_INV_GRASSLAND_BROAD = 36
ID_INV_FALLOW_BROAD = 37
ID_INV_GLOEZ_MIXED = 38
ID_INV_SPECIALTY_DISTINCT_HOPS = 39 # Tentative, only if absolutely needed
ID_INV_UNKNOWN_OR_OTHER = 0 # For truly unmapped or non-fitting INVEKOS

snar_code_to_final_id_map_broad = {
    # --- Direct Mappings to OSM Concepts (IDs 1-34) ---
    # Vineyards (OSM 23)
    901: 23, 907: 23, 906: 23, 862: 23, # WEIN, SONSTIGE WEINFLÄCHEN, SCHNITTWEINGARTEN, REBSCHULEN
    # Orchards / Fruit / Nurseries (OSM 22)
    817: 22, 818: 22, 813: 22, 814: 22, 820: 22, 854: 22, 832: 22, 844: 22, 831: 22,
    819: 22, 828: 22, 829: 22, 830: 22, 809: 22, 855: 22,
    861: 22, 860: 22, # BAUMSCHULEN (Mehrjährig, Einjährig) -> Mapped to OSM Orchard concept
    # Forestry (OSM 10)
    863: 10, 864: 10, 964: 10, 961: 10, # ENERGIEHOLZ, ERSTAUFFORSTUNG
    # Greenhouses (OSM 26)
    840: 26, 837: 26, 838: 26, 839: 26, 849: 26, 850: 26, 851: 26, 852: 26,
    845: 26, 846: 26, 848: 26, 847: 26, 843: 26, # Various tunnel/greenhouse crops

    # --- INVEKOS Extensions (New Broad Class IDs) ---

    # Most Arable Crops -> INV_AGRICULTURE_ARABLE_FIELD_CROPS (101)
    # This will be a long list of SNAR_CODEs covering cereals, roots, oilseeds, legumes, field veg, etc.
    # Example subset:
    138: ID_INV_ARABLE_BROAD, 105: ID_INV_ARABLE_BROAD, 109: ID_INV_ARABLE_BROAD, 110: ID_INV_ARABLE_BROAD,
    142: ID_INV_ARABLE_BROAD, 155: ID_INV_ARABLE_BROAD, 651: ID_INV_ARABLE_BROAD, 524: ID_INV_ARABLE_BROAD,
    308: ID_INV_ARABLE_BROAD, 751: ID_INV_ARABLE_BROAD, 307: ID_INV_ARABLE_BROAD, 301: ID_INV_ARABLE_BROAD,
    696: ID_INV_ARABLE_BROAD, # FELGEMÜSE EINKULTURIG
    679: ID_INV_ARABLE_BROAD, # SONSTIGES FELDFUTTER
    634: ID_INV_ARABLE_BROAD, # KLEEGRAS (if considered temporary/arable rotation)
    635: ID_INV_ARABLE_BROAD, # LUZERNE (if temporary)
    633: ID_INV_ARABLE_BROAD, # KLEE (if temporary)
    661: ID_INV_ARABLE_BROAD, # SONSTIGE ACKERFLÄCHEN
    671: ID_INV_ARABLE_BROAD, # SONSTIGE ACKERKULTUREN
    686: ID_INV_ARABLE_BROAD, # ELEFANTENGRAS (MISCANTHUS) - now grouped into broad arable
    # ... ALL other SNAR_CODEs that represent field-grown crops ...
    # Any combination like "WINTERGERSTE / SOJABOHNEN" (776) also goes to 101.
    # All SNAR_CODEs previously mapped to 101 in the more detailed version.

    # All Grassland Types -> INV_AGRICULTURE_GRASSLAND_ALL_TYPES (102)
    717: ID_INV_GRASSLAND_BROAD, 716: ID_INV_GRASSLAND_BROAD, 715: ID_INV_GRASSLAND_BROAD,
    992: ID_INV_GRASSLAND_BROAD, 636: ID_INV_GRASSLAND_BROAD, 642: ID_INV_GRASSLAND_BROAD,
    701: ID_INV_GRASSLAND_BROAD, 704: ID_INV_GRASSLAND_BROAD, 707: ID_INV_GRASSLAND_BROAD,
    708: ID_INV_GRASSLAND_BROAD, 710: ID_INV_GRASSLAND_BROAD, 723: ID_INV_GRASSLAND_BROAD,
    637: ID_INV_GRASSLAND_BROAD, # FUTTERGRÄSER (if permanent)
    # If Kleegras/Luzerne/Klee (633,634,635) are considered permanent grassland components, map them here instead of 101.
    # This decision depends on local agricultural practice and how they appear visually over time.
    # For simplicity, if they appear in the ARABLE list (101), they are treated as temporary.

    # All Fallow / Set Aside -> INV_AGRICULTURE_FALLOW_SET_ASIDE (103)
    771: ID_INV_FALLOW_BROAD, 721: ID_INV_FALLOW_BROAD, 653: ID_INV_FALLOW_BROAD, # FELGEMÜSE OHNE ERNTE
    959: ID_INV_FALLOW_BROAD, # 20 JÄHRIGE STILLLEGUNG

    # All GLÖZ Features (except potentially distinct buildings/monuments) -> INV_GLOEZ_MIXED_FEATURES (104)
    360: ID_INV_GLOEZ_MIXED, # GLÖZ HECKE / UFERGEHÖLZ
    359: ID_INV_GLOEZ_MIXED, # GLÖZ FELDGEHÖLZ / BAUM- / GEBÜSCHGRUPPE
    366: ID_INV_GLOEZ_MIXED, # LSE MEHRNUTZENHECKE
    361: ID_INV_GLOEZ_MIXED, # GLÖZ RAIN / BÖSCHUNG / TROCKENSTEINMAUER (base mapping)
    362: 19, # GLÖZ GRABEN / UFERRANDSTREIFEN
    365: 18, # GLÖZ TEICH / TÜMPEL
    364: ID_INV_GLOEZ_MIXED, # GLÖZ STEINRIEGEL / STEINHAGE
    # GLÖZ NATURDENKMAL FLÄCHE (363) could be 104 or remain unknown/separate if visually diverse. For now, 104.
    363: ID_INV_GLOEZ_MIXED,

    # Specialty Distinct Crop (Optional - only if Hops is critical and common)
    821: ID_INV_SPECIALTY_DISTINCT_HOPS, # HOPFEN

    # Other INVEKOS classes that might not fit neatly
    # These might need to be mapped to the closest broad category or an "Other/Unknown" INVEKOS ID
    810: ID_INV_UNKNOWN_OR_OTHER, # SONSTIGE SPEZIALKULTURFLÄCHEN
    865: ID_INV_UNKNOWN_OR_OTHER, # ANDERE DAUERKULTUREN
    767: ID_INV_ARABLE_BROAD,     # ROLLRASEN (treat as a specialized arable crop)
}

# Populate the arable group (101) with all relevant SNAR_CODEs from your CSV
# This is crucial for completeness.
all_arable_snar_codes = [
    138, 105, 109, 110, 142, 155, 111, 165, 145, 117, 118, 119, 113, 114, 112, 137, 181, 166, 157, 150, 149, 141, 146, 179, 120, 154, 758, 759,
    651, 524, 519, 525, 527, 513, 528, 631, 769,
    308, 214, 212, 213, 211, 210, 203, 626, 627, 209, 625, 624, 623, 628, 207,
    751, 307, 301, 509, 510, 658, 310, 506, 538, 539, 680, 682, 689, 690, 536, 537, 540, 772, 752, 775, 657, 302, 303, 304, 535,
    696, 768, 694, 697, 676,
    106, 115, 638, 208, 206,
    679, 634, 635, 633, # Klee, Kleegras, Luzerne if part of arable rotation
    681, 135,
    776, 184, 764, # Arable combinations
    116, # WINTERTRITICALE (from original data)
    173, # SAATMAISVERMEHRUNG
    107, # ZUCKERMAIS
    774, # DURCHWACHSENE SILPHIE
    661, 671, # Sonstige Acker...
    686, # Miscanthus/Elefantengras -> to arable
    767, # ROLLRASEN
    # All ... / FELDGEMÜSE, ... / BUCHWEIZEN, etc. combinations from your CSV
    125, 126, 129, 130, 131, 134, 139, 140, 143, 144, 148, 153, 156, 159, 160, 161, 164, 167, 168, 169, 170, 171, 174, 175, 176, 177, 178, 180, 182, 183, 185,
    204, 205, 311, 520, 526, 529, 663, 664, 665, 763, 765, 182 # Double check this last one
]
for code in all_arable_snar_codes:
    if code not in snar_code_to_final_id_map_broad: # Avoid overwriting more specific mappings like HOPFEN if it was in a similar list
        snar_code_to_final_id_map_broad[code] = ID_INV_ARABLE_BROAD


# Populate the grassland group (102)
all_grassland_snar_codes = [
    717, 716, 715, 992, 636, 642, 701, 704, 707, 708, 710, 723, 637
]
for code in all_grassland_snar_codes:
     if code not in snar_code_to_final_id_map_broad:
        snar_code_to_final_id_map_broad[code] = ID_INV_GRASSLAND_BROAD

# Ensure all SNAR_CODEs from your CSV are covered or have a default.
# The `remap_invekos_vectorized` function's `default_id` will catch any missing.

def remap_invekos_vectorized(invekos_gdf, mapping_dict, default_id):
    if 'SNAR_CODE' not in invekos_gdf.columns:
        raise ValueError("INVEKOS GeoDataFrame must contain a 'SNAR_CODE' column.")

    snar_codes_series = invekos_gdf['SNAR_CODE']
    # Ensure consistent types (e.g., if SNAR_CODE is object/str in GDF and dict keys are int)
    # This example assumes SNAR_CODE in GDF can be converted to numeric for matching int keys.
    # If SNAR_CODE contains non-numeric strings that are actual codes, adjust type handling.
    numeric_snar_codes = pd.to_numeric(snar_codes_series, errors='coerce')

    # Map, then fill NaNs (from non-numeric or unmapped codes) with default_id
    mapped_ids = numeric_snar_codes.map(mapping_dict)

    # For codes that couldn't be converted to numeric, or were not in map, assign default
    # A more robust way is to ensure all codes in the Series are in the map or explicitly handled
    invekos_gdf['FINAL_CLASS_ID'] = mapped_ids.fillna(default_id).astype(int)

    # Special handling for GLÖZ 361 (RAIN / BÖSCHUNG / TROCKENSTEINMAUER)
    if 'SNAR_BEZEICHNUNG' in invekos_gdf.columns and 361 in mapping_dict: # Check if 361 is in dict to avoid error if not
        mask_361_steinmauer = (pd.to_numeric(invekos_gdf['SNAR_CODE'], errors='coerce') == 361) & \
                              (invekos_gdf['SNAR_BEZEICHNUNG'].str.contains("TROCKENSTEINMAUER", na=False))
        # Only override if the base mapping for 361 was INV_GLOEZ_MIXED (104)
        if snar_code_to_final_id_map_broad.get(361) == ID_INV_GLOEZ_MIXED:
             invekos_gdf.loc[mask_361_steinmauer, 'FINAL_CLASS_ID'] = ID_INV_GLOEZ_MIXED # Still 104, but stone component is part of mixed.
                                                                                       # If stone needed separate: ID_INV_GLOEZ_Stone (new ID)
                                                                                       # For max 5 new classes, keeping it in MIXED is better.

    return invekos_gdf

# Define names for your FINAL BROAD class IDs
final_class_id_to_name_map_broad = {
    # OSM Classes 1-34 (use concise names)
    1: "OSM_Bld_Residential", 2: "OSM_Bld_Commercial", 3: "OSM_Bld_Industrial",
    4: "OSM_Bld_Unspecified", 5: "OSM_Road_PavedMajor", 6: "OSM_Road_PavedMinor",
    7: "OSM_Road_Unpaved", 8: "OSM_Parking", 9: "OSM_Railway", 10: "OSM_Forest",
    11: "OSM_Grassland_Generic", 12: "OSM_Heath_Scrub", 13: "OSM_BareEarth_Natural",
    14: "OSM_Wetland", 15: "OSM_Cliff", 16: "OSM_Glacier", 17: "OSM_Scree_Talus",
    18: "OSM_Water_Large", 19: "OSM_Water_Small_Linear", 20: "OSM_SwimmingPool",
    21: "OSM_Farmland_Generic", 22: "OSM_Orchard", 23: "OSM_Vineyard", 24: "OSM_Allotments",
    25: "OSM_Farmyard", 26: "OSM_Greenhouse", 27: "OSM_Util_PowerTower",
    28: "OSM_Util_WindTurbine", 29: "OSM_Util_Substation", 30: "OSM_Aeroway",
    31: "OSM_Quarry", 32: "OSM_Cemetery", 33: "OSM_Park_Recreation",
    34: "OSM_SportsPitch",
    # INVEKOS Extended Broad Classes
    ID_INV_ARABLE_BROAD: "INV_Arable_AllFieldCrops",
    ID_INV_GRASSLAND_BROAD: "INV_Grassland_AllTypes",
    ID_INV_FALLOW_BROAD: "INV_Fallow_SetAside",
    ID_INV_GLOEZ_MIXED: "INV_GLOEZ_MixedAgriEnvFeatures",
    ID_INV_SPECIALTY_DISTINCT_HOPS: "INV_SpecialtyCrop_Hops", # (If used)
    ID_INV_UNKNOWN_OR_OTHER: "INV_Other_Or_UnknownDetail"
}
# Populate the snar_code_to_final_id_map_broad with all arable and grassland codes as done previously.

In [ ]:
try:
    with rasterio.open(raster_file) as src:
        aoi_bounds_orig = src.bounds
        crs_orig = src.crs
        raster_meta_final = src.meta.copy() # For the final label raster
        transform_orig = src.transform
        width_orig = src.width
        height_orig = src.height
        aoi_polygon_orig = box(*aoi_bounds_orig)
except Exception as e:
    print(f"Error loading raster: {e}. Exiting.")
    print(e)
aoi_gdf_orig = gpd.GeoDataFrame(geometry=[aoi_polygon_orig], crs=crs_orig)

# We will clip all data to aoi_polygon_orig AT THE END of vector processing.
# Convert AOI to WGS84 for OSMnx queries
aoi_gdf_orig = gpd.GeoDataFrame(geometry=[aoi_polygon_orig], crs=crs_orig)
aoi_31287 = aoi_gdf_orig.to_crs(epsg=31287)
aoi_bbox_31287 = aoi_31287.total_bounds # Use this for ox.features_from_bbox
aoi_31256 = aoi_gdf_orig.to_crs(epsg=31256)
aoi_bbox_31256 = aoi_31256.total_bounds # Use this for ox.features_from_bbox


In [ ]:
invekos_gdf = gpd.read_file(invekos_gdf_path, bbox=tuple(aoi_bbox_31287))

In [ ]:
osm_gdf["class_id"] = osm_gdf["class_id"].astype("int")

In [ ]:
invekos_gdf_31256 = invekos_gdf.to_crs(31256)

In [ ]:
final_values = snar_code_to_final_id_map_broad.values()

In [ ]:
for final_value in list(set(final_values)):
    if final_value not in set(hierarchy):
        if final_value == 255:
            continue
        else:
            hierarchy.insert(hierarchy.index(21), final_value)


In [ ]:
len(hierarchy)

40

In [ ]:
remapped_invekos_gdf = remap_invekos_vectorized(invekos_gdf_31256, snar_code_to_final_id_map_broad, 0)

In [ ]:
np.unique(remapped_invekos_gdf["FINAL_CLASS_ID"], return_counts=True)

(array([ 0, 10, 18, 19, 22, 23, 26, 35, 36, 37, 38, 39]),
 array([  546,   429,   473,   945,  6136, 17387,  1376, 75653, 63438,
        31240,  8246,   112]))

In [ ]:
reduced_invekos_gdf = remapped_invekos_gdf[["FINAL_CLASS_ID", "geometry"]]
reduced_invekos_gdf = reduced_invekos_gdf.rename(columns={"FINAL_CLASS_ID":"class_id"})

In [ ]:
unique_classes = sorted(reduced_invekos_gdf['class_id'].unique())
class_to_id = {int(cls_name): i + 1 for i, cls_name in enumerate(unique_classes)} # Start IDs from 1
import json
with open(r"/content/drive/MyDrive/Colab Notebooks/mastarbeit/out_data\class_to_id_mapping.json", "w") as f:
    json.dump(class_to_id, f, indent=2)
print("Saved class_to_id_mapping.json")


Saved class_to_id_mapping.json


In [ ]:
def merge_and_prioritize(osm_gdf: gpd.GeoDataFrame, invekos_gdf: gpd.GeoDataFrame, osm_hierarchy: list, invekos_hierarchy: list) -> gpd.GeoDataFrame:
    """
    Performs a vectorized hierarchical overlay of two GeoDataFrames.

    The invekos_gdf is prioritized over the osm_gdf. Within each source,
    the provided hierarchy list determines the priority (lower index = higher priority).

    Args:
        osm_gdf (gpd.GeoDataFrame): GeoDataFrame from OpenStreetMap.
        invekos_gdf (gpd.GeoDataFrame): GeoDataFrame from Invekos (higher priority).
        osm_hierarchy (list): List of class_ids for OSM data, in order of priority.
        invekos_hierarchy (list): List of class_ids for Invekos data, in order of priority.

    Returns:
        gpd.GeoDataFrame: A single, non-overlapping GeoDataFrame with priorities applied.
    """
    # --- Handle empty inputs gracefully ---
    is_osm_empty = osm_gdf.empty
    is_invekos_empty = invekos_gdf.empty

    if is_osm_empty and is_invekos_empty:
        print("  Both input GDFs are empty. Returning empty GDF.")
        return gpd.GeoDataFrame(columns=['class_id', 'geometry'], crs=invekos_gdf.crs or osm_gdf.crs)
    if is_osm_empty:
        print("  OSM GDF is empty. Returning Invekos GDF.")
        return invekos_gdf
    if is_invekos_empty:
        print("  Invekos GDF is empty. Hierarchically processing OSM data only.")
        # If only OSM data is present, we still need to process its internal hierarchy.
        osm_gdf['source'] = 'osm' # Add source for consistency
        combined_gdf = osm_gdf
        full_hierarchy = [(1, class_id) for class_id in osm_hierarchy] # Priority tuple (source, class)
    else:
        # --- Prepare and Combine DataFrames ---
        invekos_gdf['source'] = 'invekos'
        osm_gdf['source'] = 'osm'
        combined_gdf = pd.concat([invekos_gdf, osm_gdf], ignore_index=True)
        # Define the full priority order: Invekos first, then OSM
        full_hierarchy = [(0, class_id) for class_id in invekos_hierarchy] + \
                         [(1, class_id) for class_id in osm_hierarchy]

    # --- Establish Priority Order ---
    priority_map = {item: i for i, item in enumerate(full_hierarchy)}
    source_map = {'invekos': 0, 'osm': 1}
    combined_gdf['priority_group'] = combined_gdf.apply(lambda row: (source_map[row['source']], row['class_id']), axis=1)
    combined_gdf['priority_rank'] = combined_gdf['priority_group'].map(priority_map)

    # Sort by the single rank column for clean iteration order
    combined_gdf = combined_gdf.sort_values('priority_rank').reset_index(drop=True)

    # --- Perform Vectorized Hierarchical Overlay ---
    final_pieces = []
    higher_priority_union = None

    # Iterate through unique priority ranks
    for rank in sorted(combined_gdf['priority_rank'].unique()):
        current_layer = combined_gdf[combined_gdf['priority_rank'] == rank]

        # Dissolve the current layer to work with a single (multi)polygon
        current_layer_dissolved = current_layer.dissolve()
        if current_layer_dissolved.empty:
            continue

        processed_geom = current_layer_dissolved.geometry.iloc[0]

        if higher_priority_union:
            processed_geom = processed_geom.difference(higher_priority_union)

        if not processed_geom.is_empty:
            class_id = current_layer['class_id'].iloc[0]
            final_pieces.append(gpd.GeoDataFrame({'class_id': [class_id], 'geometry': [processed_geom]}, crs=combined_gdf.crs))

        # Update the "eraser" with the original shape of the current layer
        current_union = current_layer.geometry.unary_union
        if higher_priority_union is None:
            higher_priority_union = current_union
        else:
            higher_priority_union = higher_priority_union.union(current_union)

    if not final_pieces:
        return gpd.GeoDataFrame(columns=['class_id', 'geometry'], crs=combined_gdf.crs)

    # --- Final Assembly ---
    final_gdf = pd.concat(final_pieces, ignore_index=True)
    # Explode MultiPolygons to ensure single geometries as required
    final_gdf = final_gdf.explode(index_parts=False, ignore_index=True)

    return final_gdf

In [ ]:
def process_lulc_data_by_subregion(
    osm_class_definitions: dict,
    osm_hierarchy: list,
    invekos_gpkg_path: str,
    invekos_hierarchy: list,
    in_raster_path: str,
    out_gdf_base_path: str,
    num_subregions_x: int = 3,
    num_subregions_y: int = 3
):
    """
    Downloads OSM data, combines it with Invekos data, and processes it hierarchically
    by splitting the AOI into subregions to manage memory and complexity.

    Args:
        osm_class_definitions: Dictionary defining OSM data to download.
        osm_hierarchy: Priority list for OSM class IDs.
        invekos_gpkg_path: Path to the high-priority Invekos GeoPackage file.
        invekos_hierarchy: Priority list for Invekos class IDs.
        in_raster_path: Path to the reference raster defining the full AOI.
        out_gdf_base_path: Base path for saving the output subregion GeoPackage files.
        num_subregions_x: Number of columns to split the AOI into.
        num_subregions_y: Number of rows to split the AOI into.
    """
    try:
        with rasterio.open(in_raster_path) as src:
            width_orig, height_orig, transform_orig, crs_orig = src.width, src.height, src.transform, src.crs
    except Exception as e:
        print(f"Error loading raster: {e}. Exiting.")
        return

    # --- Load high-priority data ONCE outside the loop for efficiency ---
    try:
        print(f"Loading Invekos data from {invekos_gpkg_path}...")
        invekos_gdf_full = gpd.read_file(invekos_gpkg_path)
        # Ensure it's in the correct CRS
        if invekos_gdf_full.crs != crs_orig:
            invekos_gdf_full = invekos_gdf_full.to_crs(crs_orig)
        print(f"Loaded {len(invekos_gdf_full)} Invekos features.")
    except Exception as e:
        print(f"Error loading Invekos data: {e}. Exiting.")
        return

    # --- Subregion Splitting Logic (same as your function) ---
    print(f"\n--- Splitting AOI into {num_subregions_y}x{num_subregions_x} subregions ---")
    subregion_width = width_orig // num_subregions_x
    subregion_height = height_orig // num_subregions_y
    subregion_aois = []
    for r in range(num_subregions_y):
        for c in range(num_subregions_x):
            win = Window(c * subregion_width, r * subregion_height, subregion_width, subregion_height)
            subregion_aois.append(gpd.GeoDataFrame(geometry=[box(*src.window_bounds(win))], crs=crs_orig))

    # --- Process Each Subregion Individually ---
    for i, aoi_gdf in enumerate(subregion_aois):
        subregion_name = f"subregion_{i+1:02d}"
        print(f"\n{'='*20} Processing {subregion_name} {'='*20}")

        aoi_wgs84 = aoi_gdf.to_crs(epsg=4326)
        aoi_bbox_wgs84 = aoi_wgs84.total_bounds

        # --- 1. Get OSM Data for the subregion ---
        osm_gdfs_for_subregion = []
        for _, params in osm_class_definitions.items():
            # (Your OSM download and buffering logic for a subregion would go here)
            # This is a simplified placeholder for the OSM download part from your original function
            try:
                gdf = ox.features.features_from_bbox(bbox=aoi_bbox_wgs84, tags=params['tags'])
                if not gdf.empty:
                    gdf_proj = gdf.to_crs(crs_orig).explode(index_parts=False, ignore_index=True)
                    gdf_proj['class_id'] = params.get("final_class_id")
                    osm_gdfs_for_subregion.append(gdf_proj[['class_id', 'geometry']])
            except Exception:
                continue # Skip if no features

        if not osm_gdfs_for_subregion:
            osm_data_gdf = gpd.GeoDataFrame(columns=['class_id', 'geometry'], crs=crs_orig)
        else:
            osm_data_gdf = pd.concat(osm_gdfs_for_subregion, ignore_index=True).pipe(gpd.GeoDataFrame)

        print(f"  Downloaded {len(osm_data_gdf)} OSM features for {subregion_name}.")

        # --- 2. Clip Invekos Data to the subregion ---
        print(f"  Clipping Invekos data to {subregion_name}...")
        reduced_invekos_gdf = gpd.clip(invekos_gdf_full, aoi_gdf)
        print(f"  Found {len(reduced_invekos_gdf)} Invekos features in {subregion_name}.")

        # --- 3. Call the core processing function ---
        print(f"  Merging and prioritizing all features for {subregion_name}...")
        final_gdf = merge_and_prioritize(
            osm_gdf=osm_data_gdf,
            invekos_gdf=reduced_invekos_gdf,
            osm_hierarchy=osm_hierarchy,
            invekos_hierarchy=invekos_hierarchy
        )

        if final_gdf.empty:
            print(f"  No features resulted from merge for {subregion_name}. Skipping output.")
            continue

        # --- 4. Final Clip and Save ---
        print(f"  Clipping final result to precise subregion boundary...")
        final_gdf = gpd.clip(final_gdf, aoi_gdf)
        final_gdf = final_gdf[~final_gdf.is_empty]
        final_gdf.geometry = final_gdf.geometry.make_valid()

        output_folder = os.path.dirname(out_gdf_base_path) or '.'
        os.makedirs(output_folder, exist_ok=True)
        subregion_out_path = os.path.join(output_folder, f"{os.path.basename(out_gdf_base_path).replace('.gpkg', '')}_{subregion_name}.gpkg")

        final_gdf.to_file(subregion_out_path, driver="GPKG")
        print(f"  SUCCESS: Saved {len(final_gdf)} final features for {subregion_name} to {subregion_out_path}")

    print("\n--- All subregions processed ---")

In [ ]:
process_lulc_data_by_subregion(
    osm_class_definitions=osm_class_definitions,
    osm_hierarchy=hierarchy,
    invekos_gpkg_path=invekos_gdf_path,
    invekos_hierarchy=hierarchy,
    in_raster_path=raster_file,
    out_gdf_base_path=r"/content/drive/MyDrive/Colab Notebooks/mastarbeit/out_data",
    num_subregions_x = 3,
    num_subregions_y = 3
)

Loading Invekos data from /content/drive/MyDrive/Colab Notebooks/mastarbeit/in_data/invekos_clipped.gpkg...
Loaded 205981 Invekos features.

--- Splitting AOI into 3x3 subregions ---

==================== Processing subregion_01 ====================
  Downloaded 107613 OSM features for subregion_01.
  Clipping Invekos data to subregion_01...
  Found 15912 Invekos features in subregion_01.
  Merging and prioritizing all features for subregion_01...
  Clipping final result to precise subregion boundary...
  SUCCESS: Saved 11301 final features for subregion_01 to /content/drive/MyDrive/Colab Notebooks/mastarbeit/out_data_subregion_01.gpkg

==================== Processing subregion_02 ====================
  Downloaded 300186 OSM features for subregion_02.
  Clipping Invekos data to subregion_02...
  Found 16823 Invekos features in subregion_02.
  Merging and prioritizing all features for subregion_02...
  Clipping final result to precise subregion boundary...
  SUCCESS: Saved 34084 final f